In [1]:
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [114]:
data=pd.read_csv("/content/drive/My Drive/NLP/Project/Input/Datafiniti_Hotel_Reviews_Jun19.csv", encoding='utf8')

data

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AWE2FvX5RxPSIh2RscTK,2018-01-18T18:43:12Z,2019-05-20T23:55:47Z,5620 Calle Real,"Hotels,Hotels and motels,Hotel and motel mgmt....",Accommodation & Food Services,Goleta,US,us/ca/goleta/5620callereal/-1127060008,34.441780,-119.819790,Best Western Plus South Coast Inn,93117,CA,2018-01-01T00:00:00.000Z,NaN,2018-01-03T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3243...,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,San Jose,UnitedStates,tatsurok2018,https://www.tripadvisor.com/Hotel_Review-g3243...,https://www.bestwestern.com/en_US/book/hotel-r...
1,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-04-02T00:00:00Z,NaN,2016-10-09T00:00:00Z,4,https://www.tripadvisor.com/Hotel_Review-g3217...,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,San Francisco,CA,STEPHEN N,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
2,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-01-06T00:00:00Z,NaN,2016-10-09T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3217...,"Parking was horrible, somebody ran into my ren...",Business,Prescott Valley,AZ,15Deborah,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
3,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-08-22T00:00:00Z,NaN,2016-10-31T00:00:00Z,5,https://www.tripadvisor.com/Hotel_Review-g3217...,Not cheap but excellent location. Price is som...,Very good,Guaynabo,PR,Wilfredo M,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
4,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-03-21T00:00:00Z,NaN,"2016-10-09T00:00:00Z,2016-03-27T00:00:00Z",2,https://www.tripadvisor.com/Hotel_Review-g3217...,If you get the room that they advertised on th...,Low chance to come back here,Reno,NV,Luc D,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotels and motels,Corporate Lodging,New...",Accommodation & Food Services,Hampton,US,us/va/hampton/3101coliseumdr/-1319580369,37.057650,-76.393310,Hampton Inn Hampton-newport News,23666,VA,2015-12-24T00:00:00Z,NaN,"2016-03-16T00:00:00Z,2016-04-03T00:00:00Z",4,http://www.tripadvisor.com/Hotel_Review-g57804...,My friends and I took a trip to Hampton for th...,Very accommodating and friendly staff!,Wallingford,CT,Tiffany1017,https://www.tripadvisor.com/Hotel_Review-g5780...,https://ad.doubleclick.net/ddm/clk/317271033;1...
9996,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotel

In [101]:
len(x1.split())

1

In [0]:
#Sklearn实现TF-IDF算法
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
class sk_TF():
  def __init__(self):
    self.vectorizer = TfidfVectorizer()

  def clean_text(self,croup):
    x=sent_tokenize(croup)
    clean_senteces=pd.Series(x).str.replace("[^a-zA-Z]"," ")
    clean_senteces=[s.lower() for s in clean_senteces]
    text_no_sp=[]
    for r in clean_senteces:
      tmp=r.split()
      sen_new=" ".join([i for i in tmp if i not in stop_words ])
      text_no_sp.append(sen_new)
    return(text_no_sp)
  def get_topic(self,clean_senteces):
    X = vectorizer.fit_transform(clean_senteces)
    terms = vectorizer.get_feature_names()
    n_pick_topics =   1            # 设定主题数为1
    lsa = TruncatedSVD(n_pick_topics)               
    X2 = lsa.fit_transform(X)
    n_pick_keywords = 5
    topic_keywords_id = [lsa.components_[t].argsort()[:-(n_pick_keywords+1):-1] for t in range(n_pick_topics)]
    for t in range(n_pick_topics):
      topic=" ".join(terms[topic_keywords_id[t][j]] for j in range(n_pick_keywords))
    return topic


In [110]:

np.seterr(divide='ignore', invalid='ignore')


tf_list=[]
for i in data['reviews.text']:
  if len(i.split())>2:
    sk=sk_TF()
    t=sk.clean_text(i)
    #print(t)
    x=sk.get_topic(t)
    #print(x)
    tf_list.append(x)
  else:
    tf_list.append("NA")
print(tf_list)

['train stay passed track near', 'king bed room comfortable clean', 'car staying somebody rental ran', 'excellent location cheap service personnel', 'stay rooms two probably one', 'adventurous eater fairly hip lends', 'hotel seemed everything recently trip', 'room numbers confirmation given quoted', 'NA', 'horrible place omg kill movies', 'well new decorated hampton inn', 'nights three barbara stayed santa', 'accommodating completely hotel kind construction', 'food good offered business promote', 'great staff rooms stay got', 'place orleans stay new expensive', 'exercise equipment broken working sorry', 'room friendly clean everyone water', 'new orleans hotel thank information', 'old hotel good location although', 'staff hotel courtis knowledgeable get', 'hotel street whitney orleans new', 'beautiful hotel suite comfortable new', 'hotel parking thank well decorated', 'hotel great well walking restaurant', 'new orleans times great good', 'thank hear giving us visit', 'guest upon told ev

In [112]:
tmp_data=data
tmp_data["tf"]=tf_list
tmp_data

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites,tf
0,AWE2FvX5RxPSIh2RscTK,2018-01-18T18:43:12Z,2019-05-20T23:55:47Z,5620 Calle Real,"Hotels,Hotels and motels,Hotel and motel mgmt....",Accommodation & Food Services,Goleta,US,us/ca/goleta/5620callereal/-1127060008,34.441780,-119.819790,Best Western Plus South Coast Inn,93117,CA,2018-01-01T00:00:00.000Z,NaN,2018-01-03T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3243...,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,San Jose,UnitedStates,tatsurok2018,https://www.tripadvisor.com/Hotel_Review-g3243...,https://www.bestwestern.com/en_US/book/hotel-r...,train stay passed track near
1,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-04-02T00:00:00Z,NaN,2016-10-09T00:00:00Z,4,https://www.tripadvisor.com/Hotel_Review-g3217...,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,San Francisco,CA,STEPHEN N,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com,king bed room comfortable clean
2,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-01-06T00:00:00Z,NaN,2016-10-09T00:00:00Z,3,https://www.tripadvisor.com/Hotel_Review-g3217...,"Parking was horrible, somebody ran into my ren...",Business,Prescott Valley,AZ,15Deborah,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com,car staying somebody rental ran
3,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-08-22T00:00:00Z,NaN,2016-10-31T00:00:00Z,5,https://www.tripadvisor.com/Hotel_Review-g3217...,Not cheap but excellent location. Price is som...,Very good,Guaynabo,PR,Wilfredo M,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com,excellent location cheap service personnel
4,AVwcj_OhkufWRAb5wi9T,2016-11-06T20:21:05Z,2019-05-20T23:31:56Z,5th And San Carlos PO Box 3574,"Hotels,Lodging,Hotel",Accommodation & Food Services,Carmel by the Sea,US,us/ca/carmelbythesea/5thandsancarlospobox3574/...,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2016-03-21T00:00:00Z,NaN,"2016-10-09T00:00:00Z,2016-03-27T00:00:00Z",2,https://www.tripadvisor.com/Hotel_Review-g3217...,If you get the room that they advertised on th...,Low chance to come back here,Reno,NV,Luc D,http://www.tripadvisor.com/Hotel_Review-g32172...,http://www.bestwestern.com,stay rooms two probably one
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwdatg0ByjofQCxo5S5,2017-04-14T05:04:33Z,2018-12-02T13:36:35Z,3101 Coliseum Dr,"Hotels,Hotels and motels,Corporate Lodging,New...",Accommodation & Food Services,Hampton,US,us/va/hampton/3101coliseumdr/-1319580369,37.057650,-76.393310,Hampton Inn Hampton-newport News,23666,VA,2015-12-24T00:00:00Z,NaN,"2016-03-16T00:00:00Z,2016-04-03T00:00:00Z",4,http://www.tripadvisor.com/Hotel_Review-g57804...,My friends and I took a trip to Hampton for th...,Very accommodating and friendly staff!,Wallingford,CT,Tiffany1017,https://www.tripadvisor.com

In [0]:
tf_list

In [17]:
x=sent_tokenize(x1)
x
clean_senteces=pd.Series(x).str.replace("[^a-zA-Z]"," ")
clean_senteces
clean_senteces=[s.lower() for s in clean_senteces]

def remove_stopwords(sen):
  sen_new=" ".join([i for i in sen if i not in stop_words ])
  return sen_new
#print(clean_senteces)
clean_senteces=[remove_stopwords(r.split())for r in clean_senteces]
clean_senteces



['stayed king suite separation bedroom living space',
 'sofa bed good back discomfort day left three night stay',
 'room clean king bed comfortable',
 'hotel located within walking distance places want']

In [0]:
grouped_hotel=data.groupby("id")['reviews.text'].apply('.'.join).reset_index()
grouped_hotel.count()
grouped_hotel

,id,reviews.text
0,AV--lQdFRxPSIh2RmtbT,I read all the ratings before I booked the Bes...
1,AV--lmxKIxWefVJwsReI,This was our second visit to this beautiful re...
2,AV--luVA3-Khe5l_ZMEJ,"Unfortunately, the hurricane cancelled our tri..."
3,AV-AN728a4HuVbedGQnJ,We stayed at this hotel since the convention w...
4,AV-Cilg2a4HuVbedGQqK,We enjoyed our stay at the Hampton Inn in Brem...
...,...,...
1428,AWdgUTtXa4HuVbedaODu,We stayed for almost two weeks but at a condo ...
1429,AWdlE0rK3-Khe5l_tDlq,"A serviceable place to stay in Waikiki, not a ..."
1430,AWfPeemaa4HuVbeddb2K,Would not stay here again. We arrived at the h...
1431,AWfaJjQba4HuVbeddiSW,"Excellent property, great breakfast Buffett - ..."


In [0]:
#sentences=sent_tokenize(grouped_hotel["reviews.text"][2])
sentences=word_tokenize(x)
len(sentences)

67

In [0]:
clean_senteces=pd.Series(sentences).str.replace("[^a-zA-Z]"," ")
clean_senteces
clean_senteces=[s.lower() for s in clean_senteces]

def remove_stopwords(sen):
  sen_new=" ".join([i for i in sen if i not in stop_words ])
  return sen_new

clean_senteces=[remove_stopwords(r.split())for r in clean_senteces]
clean_senteces

In [0]:
word_embeddings = {}
f= open("/content/drive/My Drive/NLP/Project/Input/wikiglove/glove.6B.300d.txt",encoding="utf-8")
for line in f:
  values=line.split()
  word=values[0]
  coefs=np.asarray(values[1:],dtype="float32")
  word_embeddings[word]=coefs
f.close()

In [0]:
sentence_vector=[]
for i in clean_senteces:
  if len(i)!=0:
    v=sum([word_embeddings.get(w,np.zeros((300,)))for w in i.split()])/(len(i.split())+0.001)
    
  else:
    v=np.zeros((300,))
  sentence_vector.append(v)

In [0]:
len(sentences)

67

In [0]:
sim_mat=np.zeros([len(sentences),len(sentences)])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range (len(sentences)):
  for j in range(len(sentences)):
    if i!=j:
      sim_mat[i][j]=cosine_similarity(sentence_vector[i].reshape(1,300),sentence_vector[j].reshape(1,300))[0,0]


In [0]:
import networkx as nx
nx_graph=nx.from_numpy_array(sim_mat)
score=nx.pagerank(nx_graph)

In [0]:
ranked_sentences=sorted(((score[i],s)for i,s in enumerate(sentences)),reverse=True)
for i in range(10):
  print(ranked_sentences[i][1])

room
back
stay
night
bed
bed
day
good
want
three


In [33]:
from sklearn.decomposition import TruncatedSVD           # namely LSA/LSI(即潜在语义分析)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(clean_senteces)
terms = vectorizer.get_feature_names()
print(terms)

['back', 'bed', 'bedroom', 'clean', 'comfortable', 'day', 'discomfort', 'distance', 'good', 'hotel', 'king', 'left', 'living', 'located', 'night', 'places', 'room', 'separation', 'sofa', 'space', 'stay', 'stayed', 'suite', 'three', 'walking', 'want', 'within']


In [0]:
n_pick_topics =   1            # 设定主题数为3
lsa = TruncatedSVD(n_pick_topics)               
X2 = lsa.fit_transform(X)
#n_pick_docs= 5
#topic_docs_id = [X2[:,t].argsort()[:-(n_pick_docs+1):-1] for t in range(n_pick_topics)]
#topic_docs_id

'king bed room comfortable clean'

king bed room clean comfortable
